In [1]:
# Define initial prompt and output format
initial_prompt = (
    "Please look for the following risk factor: Is the client at risk of self-harm? For instance, do they mention suicidal thoughts or ideation? Do they imply they might do physical damage to themselves or to property? Do they reference wanting to 'end it all or say it's 'not worth living'? Please output: 1. risk_type: // suicidality, 2. risk_output: // 'risk present' : this means there is evidence this risk is present in the case 'risk not present' : there is evidence the risk is NOT present or there is no evidence whether the case contains that risk or not. (If in doubt, it is better to err on the side of caution and say 'risk present') 3. explanation: // State words/terms that indicate the reason the risk_output was chosen. Be brief in your explanation. State facts found in the text, do not infer. E.g. 'Client expressed suicidal ideation'. Leave blank for 'risk not present.'"
)

output_format_prompt = (
    "Output should be STRICT JSON, containing: dictionary containing the type of risk with their output and explanation, formatted like this: {'risk_type': 'suicidality', 'risk_output': str, 'explanation': str}'"
)

In [2]:
# Define output schema
output_schema = {
    'key_to_extract': 'risk_output',
    'value_mapping': {
        'risk_present': 1,
        'risk_not_present': 0
    },
    'regex_pattern': r'"risk_output":\s*"(.*?)"'
}

In [3]:
# Set number of optimization iterations
iterations = 3

In [4]:
# Define model providers and models for evaluation and optimization
eval_provider = "ollama"
eval_model = "llama3.1"
optim_provider = "ollama"
optim_model = "llama3.1"

In [5]:
# Path to the CSV file containing review data for evaluation
eval_datapath = "risks.csv"

------------------------------------------------------------------------------------------

In [6]:
# Import necessary libraries
import pandas as pd
import sys
import os
# Add the parent directory to sys.path
# Use getcwd() to get the current working directory for Jupyter notebooks
current_dir = os.getcwd()
parent_dir = os.path.dirname(current_dir)
sys.path.append(parent_dir)
from src.iterative_prompt_optimization import optimize_prompt

In [7]:
# Load and prepare data
eval_data = pd.read_csv(eval_datapath, encoding='ISO-8859-1', usecols=['text', 'label'])
# Randomly select 50 positive and 50 negative samples
eval_data = (
    eval_data.groupby('label')
    .apply(lambda x: x.sample(n=4, random_state=42))
    .reset_index(drop=True)
)
# Shuffle the DataFrame randomly
eval_data = eval_data.sample(frac=1, random_state=42).reset_index(drop=True)
print(f"Evaluation data shape: {eval_data.shape}")
print(eval_data.head())

Evaluation data shape: (8, 2)
                                                text  label
0  Please confirm if the final balance on the acc...      0
1  Consumer said they are trying to avoid harming...      1
2  Good afternoon <person>, I refer to the compla...      0
3            Consumer had suicide attempt in January      1
4  Could you provide a gesture of goodwill for th...      0


In [8]:
# Run the prompt optimization process
best_prompt, best_metrics = optimize_prompt(
    initial_prompt,
    output_format_prompt,
    eval_data,
    iterations,
    eval_provider=eval_provider,
    eval_model=eval_model,
    optim_provider=optim_provider,
    optim_model=optim_model,
    output_schema=output_schema,
    use_cache=True  # Set to False if you want to disable caching
)
# After running the optimization process, you can analyze the results by checking 
# the generated log files in the `runs/prompt_optimization_logs_YYYYMMDD_HHMMSS` directory.

Selected evaluation provider: ollama
Selected evaluation model: llama3.1
Selected optimization provider: ollama
Selected optimization model: llama3.1
Estimated token usage: 17760
Estimated cost: $0 API Costs - Running on Local Hardware

Do you want to proceed with the optimization? (Y/N): 
Iteration 1/3


╭────────────────────────────────────────────── Current Full Prompt ──────────────────────────────────────────────╮
│ Please look for the following risk factor: Is the client at risk of self-harm? For instance, do they mention    │
│ suicidal thoughts or ideation? Do they imply they might do physical damage to themselves or to property? Do     │
│ they reference wanting to 'end it all or say it's 'not worth living'? Please output: 1. risk_type: //           │
│ suicidality, 2. risk_output: // 'risk present' : this means there is evidence this risk is present in the case  │
│ 'risk not present' : there is evidence the risk is NOT present or there is no evidence whether the case         │
│ contains that risk or not. (If in doubt, it is better to err on the side of caution and say 'risk present') 3.  │
│ explanation: // State words/terms that indicate the reason the risk_output was chosen. Be brief in your         │
│ explanation. State facts found in the text, do not infer. E.g. 'Client expressed suicidal ideation'. Leave      │
│ blank for 'risk not present.'                                                                                   │
│                                                                                                                 │
│ Output should be STRICT JSON, containing: dictionary containing the type of risk with their output and          │
│ explanation, formatted like this: {'risk_type': 'suicidality', 'risk_output': str, 'explanation': str}'         │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Processing text 1/8
Unable to find risk_output in the raw output
Prediction 1/8: None | Ground Truth: 0 🛠️ (Invalid Output Format) - Raw output: This conversation just started. We haven't discussed any previous interactions or transactions related to an account update. If you'd like to discuss a specific topic, feel free to start now!
Processing text 2/8
Prediction 2/8: 1 | Ground Truth: 1 ✅ (TP)
Processing text 3/8
Prediction 3/8: 0 | Ground Truth: 0 ✅ (TN)
Processing text 4/8
Prediction 4/8: 1 | Ground Truth: 1 ✅ (TP)
Processing text 5/8
Unable to find risk_output in the raw output
Prediction 5/8: None | Ground Truth: 0 🛠️ (Invalid Output Format) - Raw output: I'm happy to help! However, I don't have any prior interactions or commitments with you that would require a "gesture of goodwill." This conversation just started.That being said, if you'd like, I can offer a complimentary response as a way to make up for the delay in my previous processing. Please let me know what information 

Evaluation Metrics - Iteration 1
┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━┓
┃ Metric              ┃  Value ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━┩
│ Precision           │ 1.0000 │
│ Recall              │ 1.0000 │
│ Accuracy            │ 1.0000 │
│ F1-score            │ 1.0000 │
│ Valid Predictions   │      6 │
│ Invalid Predictions │      2 │
└─────────────────────┴────────┘


Analyzing misclassifications...


╭──────────────────────────────────────── Analysis of Misclassifications ─────────────────────────────────────────╮
│                                                                                                                 │
│ After analyzing the misclassifications provided, I've identified key areas where the prompt can be improved to  │
│ enhance recall while maintaining reasonable precision.                                                          │
│                                                                                                                 │
│ **False Negatives (Texts Incorrectly Classified as Negative)**                                                  │
│                                                                                                                 │
│ Upon reviewing the negative texts that were incorrectly classified as positive, I noticed the following:        │
│                                                                                                                 │
│ 1. **Vague expressions of distress**: In one text, the client mentions feeling "hopeless" and "overwhelmed,"    │
│ which might not explicitly indicate suicidal thoughts but convey a sense of desperation.                        │
│ 2. **Self-blame and guilt**: Another text contains phrases like "I'm such a failure" and "I'm to blame for      │
│ everything." While these expressions don't directly point to self-harm, they can be indicative of underlying    │
│ emotional distress.                                                                                             │
│ 3. **Fear of abandonment or rejection**: A third text shows the client expressing fear that their partner will  │
│ leave them. Although this doesn't explicitly mention suicidal thoughts, it might suggest feelings of            │
│ hopelessness or despair.                                                                                        │
│                                                                                                                 │
│ To address these cases, I recommend modifying the prompt to:                                                    │
│                                                                                                                 │
│ *   **Increase sensitivity to vague expressions of distress**: Add phrases like "expressions of hopelessness,"  │
│ "desperation," or "emotional distress" to the prompt.                                                           │
│ *   **Incorporate self-blame and guilt indicators**: Consider adding phrases like "self-blame," "guilt," or     │
│ "self-criticism" to the prompt.                                                                                 │
│ *   **Expand on fear-related expressions**: Update the prompt to include phrases like "fear of abandonment,"    │
│ "rejection," or "loss."                                                                                         │
│                                                                                                                 │
│ Here's an updated version of the prompt:                                                                        │
│                                                                                                                 │
│ Please look for the following risk factors: Is the client at risk of self-harm? For instance, do they mention   │
│ suicidal thoughts or ideation? Do they imply they might do physical damage to themselves or to property? Do     │
│ they reference wanting to 'end it all' or say it's 'not worth living'? Additionally, consider if they express:  │
│         *   Vague expressions of hopelessness or desperation                                                    │
│         *   Self-blame or guilt related statements                                                              │
│         *   Fear of abandonment or rejection          


Generating new prompt...

Iteration 2/3


╭────────────────────────────────────────────── Current Full Prompt ──────────────────────────────────────────────╮
│ To refine the prompt and improve recall while maintaining reasonable precision, I recommend the following       │
│ modifications:                                                                                                  │
│                                                                                                                 │
│ **Modified Prompt:**                                                                                            │
│                                                                                                                 │
│ Please look for the following risk factors: Is the client at risk of self-harm? For instance, do they mention   │
│ suicidal thoughts or ideation? Do they imply they might do physical damage to themselves or to property? Do     │
│ they reference wanting to 'end it all' or say it's 'not worth living'? Consider the following indicators:       │
│                                                                                                                 │
│ 1.  **Vague expressions of hopelessness or desperation**: Expressions that suggest a sense of emotional         │
│ distress, such as feeling "hopeless," "overwhelmed," or experiencing a sense of despair.                        │
│ 2.  **Self-blame or guilt-related statements**: Statements that indicate self-criticism, self-blame, or         │
│ feelings of guilt, such as saying "I'm such a failure" or expressing regret over past actions.                  │
│ 3.  **Fear of abandonment or rejection**: Expressions that suggest fear of losing someone important to them,    │
│ such as a partner, friend, or family member.                                                                    │
│                                                                                                                 │
│ If you're unsure about the presence of these risk factors, please err on the side of caution and classify the   │
│ case as having a risk present.                                                                                  │
│                                                                                                                 │
│ **Modified Output Format Instructions:**                                                                        │
│                                                                                                                 │
│ Expected output format:                                                                                         │
│ Output should be STRICT JSON, containing: dictionary containing the type of risk with their output and          │
│ explanation, formatted like this: {'risk_type': 'suicidality', 'risk_output': str, 'explanation': str}          │
│                                                                                                                 │
│ **Additional Recommendations:**                                                                                 │
│                                                                                                                 │
│ 1.  **Increase specificity for explicit denials**: Update the prompt to include phrases like "explicit denial   │
│ of suicidal thoughts" or "statements indicating no intention of self-harm."                                     │
│ 2.  **Incorporate support system indicators**: Consider adding phrases like "expressions of a strong support    │
│ system" or "statements emphasizing emotional security."                                                         │
│                                                                                                                 │
│ **Why These Changes?**                                                                                          │
│                                                       

Processing text 1/8
Unable to find risk_output in the raw output
Prediction 1/8: None | Ground Truth: 0 🛠️ (Invalid Output Format) - Raw output: To confirm the final balance update and any necessary next steps, I'll need to follow a few steps:**Step 1: Verify Account Status**I'd like to check the current status of your account. This will help me determine whether the final balance has been accurately updated.**Step 2: Review Transaction History**Next, I'll review the transaction history for any recent activity that may have affected the account balance. This could include deposits, withdrawals, or other transactions that might impact the final balance.**Step 3: Confirm Balance Update**Once I've reviewed the account status and transaction history, I can confirm whether the final balance has been updated correctly.**Further Actions Required?**If any further actions are necessary to finalize the balance update, I'll let you know. These could include:*   Reviewing and verifying recent tran